### Enviroment

In [ ]:
from tqdm import tqdm
import datetime as dt
from scipy.stats import uniform
from scipy.stats import truncnorm
from scipy.stats import bernoulli
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import mysql.connector
password_sql = 'NicholasCampeti'

### Define Drawdown Functions

In [ ]:
def drawdowns_stats(equity_curve):

    # Calcolo drawdown punto per punto
    high_watermark = equity_curve.cummax()
    drawdown_value = equity_curve / high_watermark - 1

    drawdowns = []
    durations = []
    current_dd = []

    # Identificazione dei drawdown
    for dd in drawdown_value:
        if dd < 0:
            current_dd.append(dd)
        elif current_dd:  # if current_dd --> if len(current_dd) > 0
            drawdowns.append(min(current_dd))
            durations.append(len(current_dd))
            current_dd = []

    # Se la serie termina in drawdown
    if current_dd:
        drawdowns.append(min(current_dd))
        durations.append(len(current_dd))

    # Metriche 
    max_drawdown = min(drawdowns)
    average_drawdown = sum(drawdowns) / len(drawdowns)
    average_duration = sum(durations) / len(durations)

    return {
        'Max Drawdown': max_drawdown,
        'Average Drawdown': average_drawdown,
        'Average DD Duration': average_duration}

## MNQ

### MNQ Data

In [ ]:
qqq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "qqq")

qqq_db = pd.read_sql('Select * from qqq_historical_data1 where date(date_ny) >= "2019-05-06" and date(date_ny) < "2025-12-01"', qqq_connection_sql)
print(qqq_db.dtypes)

qqq_db['date'] = qqq_db['date_ny'].dt.date
qqq_db['time'] = qqq_db['date_ny'].dt.time

qqq_db = qqq_db.sort_values(by = ['date','time']).reset_index(drop=True)


In [ ]:
qqq_buy_hold_open = qqq_db['open'].iloc[0]
qqq_buy_hold_units = round(25000 / qqq_buy_hold_open, 0)
qqq_buy_hold_close = qqq_db.groupby(qqq_db['date'])['close'].last()
qqq_buy_hold_account = 25000 + (qqq_buy_hold_close - qqq_buy_hold_open) * qqq_buy_hold_units
df_qqq_buy_hold = pd.DataFrame(list(qqq_buy_hold_account.items()), columns=['Date', 'Account'])


In [ ]:
mnq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "mnq")

mnq_db = pd.read_sql('Select * from mnq_historical_data_5m_1 where date(date_ny) >= "2015-01-01" and date(date_ny) < "2025-12-01"', mnq_connection_sql)


In [ ]:
print(mnq_db.dtypes)

mnq_db['date'] = mnq_db['date_ny'].dt.date
mnq_db['time'] = mnq_db['date_ny'].dt.time

mnq_db = mnq_db.sort_values(by = ['date','time']).reset_index(drop=True)


### MNQ Class

In [ ]:
class mnq_rdr:
    def __init__(self):
        # --- Settings ---
        self.initial_account = 25000
        self.fee = 0.25     #Per part (no roundturn)
        self.point_value = 2
        self.tick_size = 0.25
        
        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.account = []

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.dates = []
        self.unit_trade = []
        self.real_buy_price_history = []
        self.real_sell_price_history = []
        self.direction_history = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = round(min((self.account[-1] * 0.01) / (self.risk*self.point_value),(4 * self.account[-1]) / (self.opening_price*self.point_value)),0)
        if self.units > 0:
            self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, slip):
        remaining = units
        level = 0
        average_price = 0

        while remaining > 0:
            if level == 0:
                a = (5 - 10)/3 # Lim Inf Set  = 5
                initial_value = np.round(truncnorm(a, np.inf, loc = 10, scale = 3).rvs(),0)
                qty = min(initial_value, remaining)
            
            elif level == 1:
                value = round(np.round(uniform(loc = 0.6, scale = 0.3).rvs(),1) * initial_value,0)
                qty = min(value, remaining)
            
            elif level == 2:
                value = round(np.round(uniform(loc = 0.4, scale = 0.3).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            elif level >= 3:
                value = round(np.round(uniform(loc = 0.2, scale = 0.3).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            average_price += qty * exec_price
            remaining -= qty
            level += 1

        return average_price / units


    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if self.units == 0:
            self.reset()
            return

        elif direction == 'Buy':
            self.real_buy_price = self.weighted_price(self.buy_price, self.units, side='Buy', slip = self.tick_size)
            self.direction_history.append('Buy')

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(self.sell_price, self.units, side='Sell', slip = self.tick_size)
            self.direction_history.append('Sell')


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell', slip = self.tick_size
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy', slip = self.tick_size
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def starting_backtest(self,date):
        self.dates.append(date)
        self.trades_pnl.append(0)
        self.account.append(self.initial_account)
        self.real_sell_price_history.append(0)
        self.real_buy_price_history.append(0)
        self.unit_trade.append(0)
        self.direction_history.append('None')
        self.unit_list.append(0)

    def record_trade(self, date):
        pnl = ((self.real_sell_price - self.real_buy_price) * self.units * self.point_value - 2 * self.units * self.fee)

        self.dates.append(date)
        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)
        self.real_sell_price_history.append(self.real_sell_price)
        self.real_buy_price_history.append(self.real_buy_price)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)


    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    # ===============================
    # TRADE LOG
    # =============================== 

    def trade_log(self):
        return pd.DataFrame({
            "Date": pd.to_datetime(self.dates),
            "Direction": self.direction_history,
            "Average_Buy": self.real_buy_price_history,
            "Average_Sell": self.real_sell_price_history,
            "Units": self.unit_list,
            "PnL": self.trades_pnl,
            "PnL_per_Unit": self.unit_trade,
            "Account": self.account}) 

### MNQ Backtest

In [ ]:
mnq_bt = mnq_rdr()

for x in tqdm(range(0, len(mnq_db))):

    if x == 0:
        mnq_bt.starting_backtest(mnq_db['date'].iloc[x])

    # ================= ENTRY =================
    if mnq_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (mnq_db['close'].iloc[x-1] - mnq_db['open'].iloc[x-2]) > 0:
            mnq_bt.buy_pos = 1
            mnq_bt.buy_price = mnq_db['open'].iloc[x]
            mnq_bt.opening_price = mnq_bt.buy_price
            mnq_bt.stop_loss = min(mnq_db['low'].iloc[x-1], mnq_db['low'].iloc[x-2])
            mnq_bt.take_profit = (mnq_bt.opening_price - mnq_bt.stop_loss) * 10 + mnq_bt.opening_price
            mnq_bt.risk = abs(mnq_bt.opening_price - mnq_bt.stop_loss)
            mnq_bt.stop_to_be = False
            mnq_bt.stop_step_1 = False

            mnq_bt.apply_entry('Buy')
            real_risk = abs(mnq_bt.real_buy_price - mnq_bt.stop_loss)

        # SELL
        if (mnq_db['close'].iloc[x-1] - mnq_db['open'].iloc[x-2]) < 0:
            mnq_bt.sell_pos = 1
            mnq_bt.sell_price = mnq_db['open'].iloc[x]
            mnq_bt.opening_price = mnq_bt.sell_price
            mnq_bt.stop_loss = max(mnq_db['high'].iloc[x-1], mnq_db['high'].iloc[x-2])
            mnq_bt.take_profit = (mnq_bt.opening_price - mnq_bt.stop_loss) * 10 + mnq_bt.opening_price
            mnq_bt.risk = abs(mnq_bt.opening_price - mnq_bt.stop_loss)
            mnq_bt.stop_to_be = False
            mnq_bt.stop_step_1 = False

            mnq_bt.apply_entry('Sell')
            real_risk = abs(mnq_bt.real_sell_price - mnq_bt.stop_loss)


    # ================= LONG =================
    if mnq_bt.buy_pos == 1 and mnq_bt.sell_pos == 0:

        unrealized_profit = mnq_db['close'].iloc[x] - mnq_bt.real_buy_price

        # Stop loss Iniziale
        if mnq_db['low'].iloc[x] <= mnq_bt.stop_loss:
            mnq_bt.sell_pos = 1
            mnq_bt.sell_price = mnq_bt.stop_loss
            mnq_bt.apply_stop('Buy')

        # Take profit normale
        elif mnq_db['high'].iloc[x] >= mnq_bt.take_profit:
            mnq_bt.sell_pos = 1
            mnq_bt.sell_price = mnq_bt.take_profit
            mnq_bt.real_sell_price = mnq_bt.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif (not mnq_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            mnq_bt.stop_loss = mnq_bt.real_buy_price + 5 * real_risk
            mnq_bt.stop_step_1 = True
            mnq_bt.stop_to_be = True

        elif (not mnq_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            mnq_bt.stop_loss = mnq_bt.real_buy_price  # stop a BE
            mnq_bt.stop_to_be = True

    # ================= SHORT =================
    if mnq_bt.buy_pos == 0 and mnq_bt.sell_pos == 1:

        unrealized_profit = mnq_bt.real_sell_price - mnq_db['close'].iloc[x]

        # Stop loss iniziale
        if mnq_db['high'].iloc[x] >= mnq_bt.stop_loss:
            mnq_bt.buy_pos = 1
            mnq_bt.buy_price = mnq_bt.stop_loss
            mnq_bt.apply_stop('Sell')

        # Take profit normale
        elif mnq_db['low'].iloc[x] <= mnq_bt.take_profit:
            mnq_bt.buy_pos = 1
            mnq_bt.buy_price = mnq_bt.take_profit
            mnq_bt.real_buy_price = mnq_bt.buy_price  # NO slippage

        # ================= STOP MANAGEMENT ================
        elif (not mnq_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            mnq_bt.stop_loss = mnq_bt.real_sell_price - 5 * real_risk
            mnq_bt.stop_step_1 = True
            mnq_bt.stop_to_be = True

        elif (not mnq_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            mnq_bt.stop_loss = mnq_bt.real_sell_price  # stop a BE
            mnq_bt.stop_to_be = True

    # ================= END DAY =================
    if mnq_db['time'].iloc[x] == dt.time(15,55,0):

        if mnq_bt.buy_pos == 1 and mnq_bt.sell_pos == 0:
            mnq_bt.sell_pos = 1
            mnq_bt.sell_price = mnq_db['close'].iloc[x]
            mnq_bt.real_sell_price = mnq_db['close'].iloc[x]

        if mnq_bt.buy_pos == 0 and mnq_bt.sell_pos == 1:
            mnq_bt.buy_pos = 1
            mnq_bt.buy_price = mnq_db['close'].iloc[x]
            mnq_bt.real_buy_price = mnq_db['close'].iloc[x]

    # ================= RECORD =================
    if mnq_bt.buy_pos == 1 and mnq_bt.sell_pos == 1:
        mnq_bt.record_trade(mnq_db['date'].iloc[x])
        mnq_bt.reset()


### MNQ Results

In [ ]:
# Stategy Comparison
strategy = 'MNQ Trading Strategy'
Strategy_B = 'QQQ Buy & Hold'

# -----------------------------
# ----- 1. Equity Curve -------
# -----------------------------
fig_eq = go.Figure()
fig_eq.add_trace(go.Scatter(x=mnq_bt.dates, y=mnq_bt.account, mode='lines', name=f'{strategy}'))
fig_eq.add_trace(go.Scatter(x=df_qqq_buy_hold['Date'], y=df_qqq_buy_hold['Account'], mode='lines', name=f'{Strategy_B}'))

fig_eq.update_layout(
    title=dict(text="Equity Curve", yanchor="top", font=dict(size=30)),
    margin=dict(l=50, r=50, t=70, b=50),
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    yaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)", zerolinewidth=1),
    xaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)")
)
fig_eq.show()

# -----------------------------
# ----- 2. Trade Histogram -----
# -----------------------------
df_mnq_bt = mnq_bt.trade_log()
winning_mnq_bt = df_mnq_bt.loc[df_mnq_bt['PnL'] > 0]
losing_mnq_bt = df_mnq_bt.loc[df_mnq_bt['PnL'] < 0]

fig_hist = make_subplots(rows=1, cols=2, subplot_titles=("Winning Trades", "Losing Trades"))

# Istogramma vincite
fig_hist.add_trace(go.Histogram(
    x=winning_mnq_bt['PnL_per_Unit'],
    nbinsx=50,
    marker_color='lightgreen',
    opacity=0.75
), row=1, col=1)
fig_hist.add_vline(x=np.mean(winning_mnq_bt['PnL_per_Unit']),
                   line=dict(color="green", width=3, dash="dash"), row=1, col=1)

# Istogramma perdite
fig_hist.add_trace(go.Histogram(
    x=losing_mnq_bt['PnL_per_Unit'],
    nbinsx=30,
    marker_color='lightcoral',
    opacity=0.75
), row=1, col=2)
fig_hist.add_vline(x=np.mean(losing_mnq_bt['PnL_per_Unit']),
                   line=dict(color="red", width=3, dash="dash"), row=1, col=2)

fig_hist.update_layout(
    title=dict(text="Profit VS Loss per Contract", font=dict(size=48)),
    title_x=0.5,
    showlegend=False,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=1)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=2)
fig_hist.show()

# -----------------------------
# ----- 3. Winrate -------------
# -----------------------------
p_win = len(winning_mnq_bt) / len(df_mnq_bt)
p_loss = 1 - p_win

fig_winrate = go.Figure()
fig_winrate.add_trace(go.Bar(
    x=['Win', 'Loss'],
    y=[p_win, p_loss],
    marker_color=['lightgreen', 'lightcoral'],
    opacity=0.85,
    showlegend=False,
    text=[f'{p_win*100:.1f}%', f'{p_loss*100:.1f}%'],
    textposition='outside'
))
fig_winrate.update_layout(
    title=dict(text="Winrate %", font=dict(size=48)),
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_winrate.update_yaxes(range=[0, max(p_win, p_loss) + 0.1])
fig_winrate.show()

# -----------------------------
# ----- 4. Expected Value -----
# -----------------------------
EV_dollars_contract_mnq = round(np.mean(df_mnq_bt['PnL_per_Unit'].iloc[1:]), 2)
EV_perc_trade_mnq = round(np.mean(df_mnq_bt['Account'].pct_change())*100, 2)

fig_ev = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_dollars_contract_mnq,
    number={'suffix': '$', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - $ EV per Unit", 'font': {'size': 32}}
), row=1, col=1)
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_perc_trade_mnq,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % EV per Trade", 'font': {'size': 32}}
), row=1, col=2)

fig_ev.update_layout(plot_bgcolor="lightgreen", paper_bgcolor="lightgreen", font=dict(color="black"))
fig_ev.show()

# -----------------------------
# ----- 5. Sharpe & CAGR -------
# -----------------------------
perc_change_mnq = df_mnq_bt['Account'].pct_change()
sigma = np.nanstd(perc_change_mnq, ddof=1)
n_medio_trade_anno = len(df_mnq_bt[1:]) / (df_mnq_bt['Date'].dt.year.max() - df_mnq_bt['Date'].dt.year.min() + 1)
adjusted_risk_free_rate = (1 + 0.04)**(1/n_medio_trade_anno) - 1
delta_giorni = (df_mnq_bt['Date'].max() - df_mnq_bt['Date'].min()).days
anni_totali = delta_giorni / 365

sharp_ratio_mnq = ((np.nanmean(perc_change_mnq) - adjusted_risk_free_rate)/sigma) * np.sqrt(n_medio_trade_anno)
cagr = ((df_mnq_bt['Account'].iloc[-1]/df_mnq_bt['Account'].iloc[0])**(1/anni_totali)-1)*100


fig_sharpe = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=sharp_ratio_mnq,
    number={'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - Sharpe Ratio", 'font': {'size': 32}}
), row=1, col=1)
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=cagr,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % CAGR", 'font': {'size': 32}}
), row=1, col=2)

fig_sharpe.update_layout(plot_bgcolor="skyblue", paper_bgcolor="skyblue", font=dict(color="black"))
fig_sharpe.show()

# -----------------------------
# ----- 6. Rolling Average Units & Trade Count -----
# -----------------------------
df_mnq_bt_unit_list = df_mnq_bt.copy()
df_mnq_bt_unit_list['Rolling_Avg'] = df_mnq_bt_unit_list['Units'].iloc[1:].rolling(window=20).mean()
n_trades = len(df_mnq_bt.iloc[1:])

fig_units = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], specs=[[{"type": "xy"}, {"type": "domain"}]], subplot_titles=("Average Traded Units (20)", ""))

# Rolling average line
fig_units.add_trace(go.Scatter(
    x=df_mnq_bt_unit_list['Date'].iloc[20:], 
    y=df_mnq_bt_unit_list['Rolling_Avg'].iloc[20:], 
    mode='lines', 
    name=f'{strategy}'
), row=1, col=1)

# Number of trades indicator
fig_units.add_trace(go.Indicator(
    mode="number",
    value=n_trades,
    number={'valueformat': '.0f', 'font': {'size': 48}},
    title={'text': "Number of Trades", 'font': {'size': 32}}
), row=1, col=2)

fig_units.update_layout(plot_bgcolor="rgba(64,64,64,0.8)", paper_bgcolor="rgba(64,64,64,0.8)", font=dict(color="white"))
fig_units.show()

# -----------------------------
# ----- 7. Drawdown Statistics -----
# -----------------------------
equity_curve_mnq_bt = df_mnq_bt['Account']
stats_A = drawdowns_stats(equity_curve_mnq_bt)

equity_curve_qqq_buy_hold = df_qqq_buy_hold['Account']
stats_B = drawdowns_stats(equity_curve_qqq_buy_hold)

fig_dd = go.Figure()

fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{strategy} - Max DD", 'font':{'size':32}}, domain={'row':0,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':0,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':0,'column':2}))

fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{Strategy_B} - Max DD", 'font':{'size':32}}, domain={'row':1,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':1,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':1,'column':2}))

fig_dd.update_layout(
    grid={'rows':2,'columns':3},
    title=dict(text="Drawdown Statistics", font=dict(size=48)),
    height=500,
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white")
)

fig_dd.add_shape(
    type="line",
    x0=0, x1=1,       
    y0=0.55, y1=0.55,  
    xref="paper",
    yref="paper",
    line=dict(
        color="white",
        width=2))

fig_dd.show()


## NQ

### NQ Data

In [ ]:
qqq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "qqq")

qqq_db = pd.read_sql('Select * from qqq_historical_data1 where date(date_ny) >= "2015-01-01" and date(date_ny) < "2025-12-01"', qqq_connection_sql)
print(qqq_db.dtypes)

qqq_db['date'] = qqq_db['date_ny'].dt.date
qqq_db['time'] = qqq_db['date_ny'].dt.time

qqq_db = qqq_db.sort_values(by = ['date','time']).reset_index(drop=True)


In [ ]:
qqq_buy_hold_open = qqq_db['open'].iloc[0]
qqq_buy_hold_units = round(100000 / qqq_buy_hold_open, 0)
qqq_buy_hold_close = qqq_db.groupby(qqq_db['date'])['close'].last()
qqq_buy_hold_account = 100000 + (qqq_buy_hold_close - qqq_buy_hold_open) * qqq_buy_hold_units
df_qqq_buy_hold = pd.DataFrame(list(qqq_buy_hold_account.items()), columns=['Date', 'Account'])


In [ ]:
nq_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "nq")

nq_db = pd.read_sql('Select * from nq_historical_data_5m_1 where date(date_ny) >= "2015-01-01" and date(date_ny) < "2025-12-01"', nq_connection_sql)


In [ ]:
print(nq_db.dtypes)

nq_db['date'] = nq_db['date_ny'].dt.date
nq_db['time'] = nq_db['date_ny'].dt.time

nq_db = nq_db.sort_values(by = ['date','time']).reset_index(drop=True)


### NQ Class

In [ ]:
class nq_rdr:
    def __init__(self):
        # --- Settings ---
        self.initial_account = 100000
        self.fee = 0.85     #Per part (no roundturn)
        self.point_value = 20
        self.tick_size = 0.25
        
        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.account = []

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.dates = []
        self.unit_trade = []
        self.real_buy_price_history = []
        self.real_sell_price_history = []
        self.direction_history = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = round(min((self.account[-1] * 0.01) / (self.risk*self.point_value),(4 * self.account[-1]) / (self.opening_price*self.point_value)),0)
        if self.units > 0:
            self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, slip):
        remaining = units
        level = 0
        average_price = 0

        while remaining > 0:
            if level == 0:
                a = (5 - 12)/4 # Lim Inf Set  = 5
                initial_value = np.round(truncnorm(a, np.inf, loc = 12, scale = 4).rvs(),0)
                qty = min(initial_value, remaining)
            
            elif level == 1:
                value = round(np.round(uniform(loc = 0.6, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)
            
            elif level == 2:
                value = round(np.round(uniform(loc = 0.4, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            elif level >= 3:
                value = round(np.round(uniform(loc = 0.2, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            average_price += qty * exec_price
            remaining -= qty
            level += 1

        return average_price / units


    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if self.units == 0:
            self.reset()
            return

        elif direction == 'Buy':
            self.real_buy_price = self.weighted_price(self.buy_price, self.units, side='Buy', slip = self.tick_size)
            self.direction_history.append('Buy')

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(self.sell_price, self.units, side='Sell', slip = self.tick_size)
            self.direction_history.append('Sell')


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell', slip = self.tick_size
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy', slip = self.tick_size
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def starting_backtest(self,date):
        self.dates.append(date)
        self.trades_pnl.append(0)
        self.account.append(self.initial_account)
        self.real_sell_price_history.append(0)
        self.real_buy_price_history.append(0)
        self.unit_trade.append(0)
        self.direction_history.append('None')
        self.unit_list.append(0)

    def record_trade(self, date):
        pnl = ((self.real_sell_price - self.real_buy_price) * self.units * self.point_value - 2 * self.units * self.fee)

        self.dates.append(date)
        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)
        self.real_sell_price_history.append(self.real_sell_price)
        self.real_buy_price_history.append(self.real_buy_price)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)


    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    # ===============================
    # TRADE LOG
    # =============================== 

    def trade_log(self):
        return pd.DataFrame({
            "Date": pd.to_datetime(self.dates),
            "Direction": self.direction_history,
            "Average_Buy": self.real_buy_price_history,
            "Average_Sell": self.real_sell_price_history,
            "Units": self.unit_list,
            "PnL": self.trades_pnl,
            "PnL_per_Unit": self.unit_trade,
            "Account": self.account}) 

### NQ Backtest

In [ ]:
nq_bt = nq_rdr()

for x in tqdm(range(0, len(nq_db))):

    if x == 0:
        nq_bt.starting_backtest(nq_db['date'].iloc[x])

    # ================= ENTRY =================
    if nq_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (nq_db['close'].iloc[x-1] - nq_db['open'].iloc[x-2]) > 0:
            nq_bt.buy_pos = 1
            nq_bt.buy_price = nq_db['open'].iloc[x]
            nq_bt.opening_price = nq_bt.buy_price
            nq_bt.stop_loss = min(nq_db['low'].iloc[x-1], nq_db['low'].iloc[x-2])
            nq_bt.take_profit = (nq_bt.opening_price - nq_bt.stop_loss) * 10 + nq_bt.opening_price
            nq_bt.risk = abs(nq_bt.opening_price - nq_bt.stop_loss)
            nq_bt.stop_to_be = False
            nq_bt.stop_step_1 = False

            nq_bt.apply_entry('Buy')
            real_risk = abs(nq_bt.real_buy_price - nq_bt.stop_loss)

        # SELL
        if (nq_db['close'].iloc[x-1] - nq_db['open'].iloc[x-2]) < 0:
            nq_bt.sell_pos = 1
            nq_bt.sell_price = nq_db['open'].iloc[x]
            nq_bt.opening_price = nq_bt.sell_price
            nq_bt.stop_loss = max(nq_db['high'].iloc[x-1], nq_db['high'].iloc[x-2])
            nq_bt.take_profit = (nq_bt.opening_price - nq_bt.stop_loss) * 10 + nq_bt.opening_price
            nq_bt.risk = abs(nq_bt.opening_price - nq_bt.stop_loss)
            nq_bt.stop_to_be = False
            nq_bt.stop_step_1 = False

            nq_bt.apply_entry('Sell')
            real_risk = abs(nq_bt.real_sell_price - nq_bt.stop_loss)


    # ================= LONG =================
    if nq_bt.buy_pos == 1 and nq_bt.sell_pos == 0:

        unrealized_profit = nq_db['close'].iloc[x] - nq_bt.real_buy_price

        # Stop loss Iniziale
        if nq_db['low'].iloc[x] <= nq_bt.stop_loss:
            nq_bt.sell_pos = 1
            nq_bt.sell_price = nq_bt.stop_loss
            nq_bt.apply_stop('Buy')

        # Take profit normale
        elif nq_db['high'].iloc[x] >= nq_bt.take_profit:
            nq_bt.sell_pos = 1
            nq_bt.sell_price = nq_bt.take_profit
            nq_bt.real_sell_price = nq_bt.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif (not nq_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            nq_bt.stop_loss = nq_bt.real_buy_price + 5 * real_risk
            nq_bt.stop_step_1 = True
            nq_bt.stop_to_be = True

        elif (not nq_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            nq_bt.stop_loss = nq_bt.real_buy_price  # stop a BE
            nq_bt.stop_to_be = True

    # ================= SHORT =================
    if nq_bt.buy_pos == 0 and nq_bt.sell_pos == 1:

        unrealized_profit = nq_bt.real_sell_price - nq_db['close'].iloc[x]

        # Stop loss iniziale
        if nq_db['high'].iloc[x] >= nq_bt.stop_loss:
            nq_bt.buy_pos = 1
            nq_bt.buy_price = nq_bt.stop_loss
            nq_bt.apply_stop('Sell')

        # Take profit normale
        elif nq_db['low'].iloc[x] <= nq_bt.take_profit:
            nq_bt.buy_pos = 1
            nq_bt.buy_price = nq_bt.take_profit
            nq_bt.real_buy_price = nq_bt.buy_price  # NO slippage

        # ================= STOP MANAGEMENT ================
        elif (not nq_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            nq_bt.stop_loss = nq_bt.real_sell_price - 5 * real_risk
            nq_bt.stop_step_1 = True
            nq_bt.stop_to_be = True

        elif (not nq_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            nq_bt.stop_loss = nq_bt.real_sell_price  # stop a BE
            nq_bt.stop_to_be = True

    # ================= END DAY =================
    if nq_db['time'].iloc[x] == dt.time(15,55,0):

        if nq_bt.buy_pos == 1 and nq_bt.sell_pos == 0:
            nq_bt.sell_pos = 1
            nq_bt.sell_price = nq_db['close'].iloc[x]
            nq_bt.real_sell_price = nq_db['close'].iloc[x]

        if nq_bt.buy_pos == 0 and nq_bt.sell_pos == 1:
            nq_bt.buy_pos = 1
            nq_bt.buy_price = nq_db['close'].iloc[x]
            nq_bt.real_buy_price = nq_db['close'].iloc[x]

    # ================= RECORD =================
    if nq_bt.buy_pos == 1 and nq_bt.sell_pos == 1:
        nq_bt.record_trade(nq_db['date'].iloc[x])
        nq_bt.reset()


### NQ Results

In [ ]:
# Stategy Comparison
strategy = 'NQ Trading Strategy'
Strategy_B = 'QQQ Buy & Hold'

# -----------------------------
# ----- 1. Equity Curve -------
# -----------------------------
fig_eq = go.Figure()
fig_eq.add_trace(go.Scatter(x=nq_bt.dates, y=nq_bt.account, mode='lines', name=f'{strategy}'))
fig_eq.add_trace(go.Scatter(x=df_qqq_buy_hold['Date'], y=df_qqq_buy_hold['Account'], mode='lines', name=f'{Strategy_B}'))

fig_eq.update_layout(
    title=dict(text="Equity Curve", yanchor="top", font=dict(size=30)),
    margin=dict(l=50, r=50, t=70, b=50),
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    yaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)", zerolinewidth=1),
    xaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)")
)
fig_eq.show()

# -----------------------------
# ----- 2. Trade Histogram -----
# -----------------------------
df_nq_bt = nq_bt.trade_log()
winning_nq_bt = df_nq_bt.loc[df_nq_bt['PnL'] > 0]
losing_nq_bt = df_nq_bt.loc[df_nq_bt['PnL'] < 0]

fig_hist = make_subplots(rows=1, cols=2, subplot_titles=("Winning Trades", "Losing Trades"))

# Istogramma vincite
fig_hist.add_trace(go.Histogram(
    x=winning_nq_bt['PnL_per_Unit'],
    nbinsx=50,
    marker_color='lightgreen',
    opacity=0.75
), row=1, col=1)
fig_hist.add_vline(x=np.mean(winning_nq_bt['PnL_per_Unit']),
                   line=dict(color="green", width=3, dash="dash"), row=1, col=1)

# Istogramma perdite
fig_hist.add_trace(go.Histogram(
    x=losing_nq_bt['PnL_per_Unit'],
    nbinsx=30,
    marker_color='lightcoral',
    opacity=0.75
), row=1, col=2)
fig_hist.add_vline(x=np.mean(losing_nq_bt['PnL_per_Unit']),
                   line=dict(color="red", width=3, dash="dash"), row=1, col=2)

fig_hist.update_layout(
    title=dict(text="Profit VS Loss per Contract", font=dict(size=48)),
    title_x=0.5,
    showlegend=False,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=1)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=2)
fig_hist.show()

# -----------------------------
# ----- 3. Winrate -------------
# -----------------------------
p_win = len(winning_nq_bt) / len(df_nq_bt)
p_loss = 1 - p_win

fig_winrate = go.Figure()
fig_winrate.add_trace(go.Bar(
    x=['Win', 'Loss'],
    y=[p_win, p_loss],
    marker_color=['lightgreen', 'lightcoral'],
    opacity=0.85,
    showlegend=False,
    text=[f'{p_win*100:.1f}%', f'{p_loss*100:.1f}%'],
    textposition='outside'
))
fig_winrate.update_layout(
    title=dict(text="Winrate %", font=dict(size=48)),
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_winrate.update_yaxes(range=[0, max(p_win, p_loss) + 0.1])
fig_winrate.show()

# -----------------------------
# ----- 4. Expected Value -----
# -----------------------------
EV_dollars_contract_nq = round(np.mean(df_nq_bt['PnL_per_Unit'].iloc[1:]), 2)
EV_perc_trade_nq = round(np.mean(df_nq_bt['Account'].pct_change())*100, 2)

fig_ev = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_dollars_contract_nq,
    number={'suffix': '$', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - $ EV per Unit", 'font': {'size': 32}}
), row=1, col=1)
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_perc_trade_nq,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % EV per Trade", 'font': {'size': 32}}
), row=1, col=2)

fig_ev.update_layout(plot_bgcolor="lightgreen", paper_bgcolor="lightgreen", font=dict(color="black"))
fig_ev.show()

# -----------------------------
# ----- 5. Sharpe & CAGR -------
# -----------------------------
perc_change_nq = df_nq_bt['Account'].pct_change()
sigma = np.nanstd(perc_change_nq, ddof=1)
n_medio_trade_anno = len(df_nq_bt[1:]) / (df_nq_bt['Date'].dt.year.max() - df_nq_bt['Date'].dt.year.min() + 1)
adjusted_risk_free_rate = (1 + 0.04)**(1/n_medio_trade_anno) - 1
delta_giorni = (df_nq_bt['Date'].max() - df_nq_bt['Date'].min()).days
anni_totali = delta_giorni / 365

sharp_ratio_nq = ((np.nanmean(perc_change_nq) - adjusted_risk_free_rate)/sigma) * np.sqrt(n_medio_trade_anno)
cagr = ((df_nq_bt['Account'].iloc[-1]/df_nq_bt['Account'].iloc[0])**(1/anni_totali)-1)*100

fig_sharpe = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=sharp_ratio_nq,
    number={'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - Sharpe Ratio", 'font': {'size': 32}}
), row=1, col=1)
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=cagr,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % CAGR", 'font': {'size': 32}}
), row=1, col=2)

fig_sharpe.update_layout(plot_bgcolor="skyblue", paper_bgcolor="skyblue", font=dict(color="black"))
fig_sharpe.show()

# -----------------------------
# ----- 6. Rolling Average Units & Trade Count -----
# -----------------------------
df_nq_bt_unit_list = df_nq_bt.copy()
df_nq_bt_unit_list['Rolling_Avg'] = df_nq_bt_unit_list['Units'].iloc[1:].rolling(window=20).mean()
n_trades = len(df_nq_bt.iloc[1:])

fig_units = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], specs=[[{"type": "xy"}, {"type": "domain"}]], subplot_titles=("Average Traded Units (20)", ""))

# Rolling average line
fig_units.add_trace(go.Scatter(
    x=df_nq_bt_unit_list['Date'].iloc[20:], 
    y=df_nq_bt_unit_list['Rolling_Avg'].iloc[20:], 
    mode='lines', 
    name=f'{strategy}'
), row=1, col=1)

# Number of trades indicator
fig_units.add_trace(go.Indicator(
    mode="number",
    value=n_trades,
    number={'valueformat': '.0f', 'font': {'size': 48}},
    title={'text': "Number of Trades", 'font': {'size': 32}}
), row=1, col=2)

fig_units.update_layout(plot_bgcolor="rgba(64,64,64,0.8)", paper_bgcolor="rgba(64,64,64,0.8)", font=dict(color="white"))
fig_units.show()

# -----------------------------
# ----- 7. Drawdown Statistics -----
# -----------------------------
equity_curve_nq_bt = df_nq_bt['Account']
stats_A = drawdowns_stats(equity_curve_nq_bt)

equity_curve_qqq_buy_hold = df_qqq_buy_hold['Account']
stats_B = drawdowns_stats(equity_curve_qqq_buy_hold)

fig_dd = go.Figure()

# Strategia A
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{strategy} - Max DD", 'font':{'size':32}}, domain={'row':0,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':0,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':0,'column':2}))

# Strategia B
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{Strategy_B} - Max DD", 'font':{'size':32}}, domain={'row':1,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':1,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':1,'column':2}))

fig_dd.update_layout(
    grid={'rows':2,'columns':3},
    title=dict(text="Drawdown Statistics", font=dict(size=48)),
    height=500,
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white")
)

fig_dd.add_shape(
    type="line",
    x0=0, x1=1,       
    y0=0.55, y1=0.55,  
    xref="paper",
    yref="paper",
    line=dict(
        color="white",
        width=2))

fig_dd.show()


## ES

### ES Data

In [ ]:
spy_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "spy")

spy_db = pd.read_sql('Select * from spy_historical_data1 where date(date_ny) >= "2015-01-01" and date(date_ny) < "2025-12-01"', spy_connection_sql)
print(spy_db.dtypes)

spy_db['date'] = spy_db['date_ny'].dt.date
spy_db['time'] = spy_db['date_ny'].dt.time

spy_db = spy_db.sort_values(by = ['date','time']).reset_index(drop=True) 

In [ ]:
spy_buy_hold_open = spy_db['open'].iloc[0]
spy_buy_hold_units = round(100000 / spy_buy_hold_open, 0)
spy_buy_hold_close = spy_db.groupby(spy_db['date'])['close'].last()
spy_buy_hold_account = 100000 + (spy_buy_hold_close - spy_buy_hold_open) * spy_buy_hold_units
df_spy_buy_hold = pd.DataFrame(list(spy_buy_hold_account.items()), columns=['Date', 'Account'])

In [ ]:
es_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "es")

es_db = pd.read_sql('Select * from es_historical_data_5m_1 where date(date_ny) >= "2015-01-01" and date(date_ny) < "2025-12-01"', es_connection_sql) 

In [ ]:
print(es_db.dtypes)

es_db['date'] = es_db['date_ny'].dt.date
es_db['time'] = es_db['date_ny'].dt.time

es_db = es_db.sort_values(by = ['date','time']).reset_index(drop=True)

### ES Class

In [ ]:
class es_rdr:
    def __init__(self):
        # --- Settings ---
        self.initial_account = 100000
        self.fee = 0.85     #Per part (no roundturn)
        self.point_value = 50
        self.tick_size = 0.25
        
        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.account = []

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.dates = []
        self.unit_trade = []
        self.real_buy_price_history = []
        self.real_sell_price_history = []
        self.direction_history = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = round(min((self.account[-1] * 0.01) / (self.risk*self.point_value),(4 * self.account[-1]) / (self.opening_price*self.point_value)),0)
        if self.units > 0:
            self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, slip):
        remaining = units
        level = 0
        average_price = 0

        while remaining > 0:
            if level == 0:
                a = (5 - 12)/4 # Lim Inf Set  = 5
                initial_value = np.round(truncnorm(a, np.inf, loc = 12, scale = 4).rvs(),0)
                qty = min(initial_value, remaining)
            
            elif level == 1:
                value = round(np.round(uniform(loc = 0.6, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)
            
            elif level == 2:
                value = round(np.round(uniform(loc = 0.4, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            elif level >= 3:
                value = round(np.round(uniform(loc = 0.2, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            average_price += qty * exec_price
            remaining -= qty
            level += 1

        return average_price / units

    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if self.units == 0:
            self.reset()
            return

        elif direction == 'Buy':
            self.real_buy_price = self.weighted_price(self.buy_price, self.units, side='Buy', slip = self.tick_size)
            self.direction_history.append('Buy')

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(self.sell_price, self.units, side='Sell', slip = self.tick_size)
            self.direction_history.append('Sell')


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell', slip = self.tick_size
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy', slip = self.tick_size
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def starting_backtest(self,date):
        self.dates.append(date)
        self.trades_pnl.append(0)
        self.account.append(self.initial_account)
        self.real_sell_price_history.append(0)
        self.real_buy_price_history.append(0)
        self.unit_trade.append(0)
        self.direction_history.append('None')
        self.unit_list.append(0)

    def record_trade(self, date):
        pnl = ((self.real_sell_price - self.real_buy_price) * self.units * self.point_value - 2 * self.units * self.fee)

        self.dates.append(date)
        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)
        self.real_sell_price_history.append(self.real_sell_price)
        self.real_buy_price_history.append(self.real_buy_price)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)


    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    # ===============================
    # TRADE LOG
    # =============================== 

    def trade_log(self):
        return pd.DataFrame({
            "Date": pd.to_datetime(self.dates),
            "Direction": self.direction_history,
            "Average_Buy": self.real_buy_price_history,
            "Average_Sell": self.real_sell_price_history,
            "Units": self.unit_list,
            "PnL": self.trades_pnl,
            "PnL_per_Unit": self.unit_trade,
            "Account": self.account}) 

### ES Backtest

In [ ]:
es_bt = es_rdr()

for x in tqdm(range(0, len(es_db))):

    if x == 0:
        es_bt.starting_backtest(es_db['date'].iloc[x])

    # ================= ENTRY =================
    if es_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (es_db['close'].iloc[x-1] - es_db['open'].iloc[x-2]) > 0:
            es_bt.buy_pos = 1
            es_bt.buy_price = es_db['open'].iloc[x]
            es_bt.opening_price = es_bt.buy_price
            es_bt.stop_loss = min(es_db['low'].iloc[x-1], es_db['low'].iloc[x-2])
            es_bt.take_profit = (es_bt.opening_price - es_bt.stop_loss) * 10 + es_bt.opening_price
            es_bt.risk = abs(es_bt.opening_price - es_bt.stop_loss)
            es_bt.stop_to_be = False
            es_bt.stop_step_1 = False

            es_bt.apply_entry('Buy')
            real_risk = abs(es_bt.real_buy_price - es_bt.stop_loss)

        # SELL
        if (es_db['close'].iloc[x-1] - es_db['open'].iloc[x-2]) < 0:
            es_bt.sell_pos = 1
            es_bt.sell_price = es_db['open'].iloc[x]
            es_bt.opening_price = es_bt.sell_price
            es_bt.stop_loss = max(es_db['high'].iloc[x-1], es_db['high'].iloc[x-2])
            es_bt.take_profit = (es_bt.opening_price - es_bt.stop_loss) * 10 + es_bt.opening_price
            es_bt.risk = abs(es_bt.opening_price - es_bt.stop_loss)
            es_bt.stop_to_be = False
            es_bt.stop_step_1 = False

            es_bt.apply_entry('Sell')
            real_risk = abs(es_bt.real_sell_price - es_bt.stop_loss)


    # ================= LONG =================
    if es_bt.buy_pos == 1 and es_bt.sell_pos == 0:

        unrealized_profit = es_db['close'].iloc[x] - es_bt.real_buy_price

        # Stop loss Iniziale
        if es_db['low'].iloc[x] <= es_bt.stop_loss:
            es_bt.sell_pos = 1
            es_bt.sell_price = es_bt.stop_loss
            es_bt.apply_stop('Buy')

        # Take profit normale
        elif es_db['high'].iloc[x] >= es_bt.take_profit:
            es_bt.sell_pos = 1
            es_bt.sell_price = es_bt.take_profit
            es_bt.real_sell_price = es_bt.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif (not es_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            es_bt.stop_loss = es_bt.real_buy_price + 5 * real_risk
            es_bt.stop_step_1 = True
            es_bt.stop_to_be = True

        elif (not es_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            es_bt.stop_loss = es_bt.real_buy_price  # stop a BE
            es_bt.stop_to_be = True

    # ================= SHORT =================
    if es_bt.buy_pos == 0 and es_bt.sell_pos == 1:

        unrealized_profit = es_bt.real_sell_price - es_db['close'].iloc[x]

        # Stop loss iniziale
        if es_db['high'].iloc[x] >= es_bt.stop_loss:
            es_bt.buy_pos = 1
            es_bt.buy_price = es_bt.stop_loss
            es_bt.apply_stop('Sell')

        # Take profit normale
        elif es_db['low'].iloc[x] <= es_bt.take_profit:
            es_bt.buy_pos = 1
            es_bt.buy_price = es_bt.take_profit
            es_bt.real_buy_price = es_bt.buy_price  # NO slippage

        # ================= STOP MANAGEMENT ================
        elif (not es_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            es_bt.stop_loss = es_bt.real_sell_price - 5 * real_risk
            es_bt.stop_step_1 = True
            es_bt.stop_to_be = True

        elif (not es_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            es_bt.stop_loss = es_bt.real_sell_price  # stop a BE
            es_bt.stop_to_be = True

    # ================= END DAY =================
    if es_db['time'].iloc[x] == dt.time(15,55,0):

        if es_bt.buy_pos == 1 and es_bt.sell_pos == 0:
            es_bt.sell_pos = 1
            es_bt.sell_price = es_db['close'].iloc[x]
            es_bt.real_sell_price = es_db['close'].iloc[x]

        if es_bt.buy_pos == 0 and es_bt.sell_pos == 1:
            es_bt.buy_pos = 1
            es_bt.buy_price = es_db['close'].iloc[x]
            es_bt.real_buy_price = es_db['close'].iloc[x]

    # ================= RECORD =================
    if es_bt.buy_pos == 1 and es_bt.sell_pos == 1:
        es_bt.record_trade(es_db['date'].iloc[x])
        es_bt.reset()

### ES Results

In [ ]:
# Stategy Comparison
strategy = 'ES Trading Strategy'
Strategy_B = 'SPY Buy & Hold'

# -----------------------------
# ----- 1. Equity Curve -------
# -----------------------------
fig_eq = go.Figure()
fig_eq.add_trace(go.Scatter(x=es_bt.dates, y=es_bt.account, mode='lines', name=f'{strategy}'))
fig_eq.add_trace(go.Scatter(x=df_spy_buy_hold['Date'], y=df_spy_buy_hold['Account'], mode='lines', name=f'{Strategy_B}'))

fig_eq.update_layout(
    title=dict(text="Equity Curve", yanchor="top", font=dict(size=30)),
    margin=dict(l=50, r=50, t=70, b=50),
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    yaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)", zerolinewidth=1),
    xaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)")
)
fig_eq.show()

# -----------------------------
# ----- 2. Trade Histogram -----
# -----------------------------
df_es_bt = es_bt.trade_log()
winning_es_bt = df_es_bt.loc[df_es_bt['PnL'] > 0]
losing_es_bt = df_es_bt.loc[df_es_bt['PnL'] < 0]

fig_hist = make_subplots(rows=1, cols=2, subplot_titles=("Winning Trades", "Losing Trades"))

# Istogramma vincite
fig_hist.add_trace(go.Histogram(
    x=winning_es_bt['PnL_per_Unit'],
    nbinsx=50,
    marker_color='lightgreen',
    opacity=0.75
), row=1, col=1)
fig_hist.add_vline(x=np.mean(winning_es_bt['PnL_per_Unit']),
                   line=dict(color="green", width=3, dash="dash"), row=1, col=1)

# Istogramma perdite
fig_hist.add_trace(go.Histogram(
    x=losing_es_bt['PnL_per_Unit'],
    nbinsx=30,
    marker_color='lightcoral',
    opacity=0.75
), row=1, col=2)
fig_hist.add_vline(x=np.mean(losing_es_bt['PnL_per_Unit']),
                   line=dict(color="red", width=3, dash="dash"), row=1, col=2)

fig_hist.update_layout(
    title=dict(text="Profit VS Loss per Contract", font=dict(size=48)),
    title_x=0.5,
    showlegend=False,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=1)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=2)
fig_hist.show()

# -----------------------------
# ----- 3. Winrate -------------
# -----------------------------
p_win = len(winning_es_bt) / len(df_es_bt)
p_loss = 1 - p_win

fig_winrate = go.Figure()
fig_winrate.add_trace(go.Bar(
    x=['Win', 'Loss'],
    y=[p_win, p_loss],
    marker_color=['lightgreen', 'lightcoral'],
    opacity=0.85,
    showlegend=False,
    text=[f'{p_win*100:.1f}%', f'{p_loss*100:.1f}%'],
    textposition='outside'
))
fig_winrate.update_layout(
    title=dict(text="Winrate %", font=dict(size=48)),
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_winrate.update_yaxes(range=[0, max(p_win, p_loss) + 0.1])
fig_winrate.show()

# -----------------------------
# ----- 4. Expected Value -----
# -----------------------------
EV_dollars_contract_es = round(np.mean(df_es_bt['PnL_per_Unit'].iloc[1:]), 2)
EV_perc_trade_es = round(np.mean(df_es_bt['Account'].pct_change())*100, 2)

fig_ev = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_dollars_contract_es,
    number={'suffix': '$', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - $ EV per Unit", 'font': {'size': 32}}
), row=1, col=1)
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_perc_trade_es,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % EV per Trade", 'font': {'size': 32}}
), row=1, col=2)

fig_ev.update_layout(plot_bgcolor="lightgreen", paper_bgcolor="lightgreen", font=dict(color="black"))
fig_ev.show()

# -----------------------------
# ----- 5. Sharpe & CAGR -------
# -----------------------------
perc_change_es = df_es_bt['Account'].pct_change()
sigma = np.nanstd(perc_change_es, ddof=1)
n_medio_trade_anno = len(df_es_bt[1:]) / (df_es_bt['Date'].dt.year.max() - df_es_bt['Date'].dt.year.min() + 1)
adjusted_risk_free_rate = (1 + 0.04)**(1/n_medio_trade_anno) - 1
delta_giorni = (df_es_bt['Date'].max() - df_es_bt['Date'].min()).days
anni_totali = delta_giorni / 365

sharp_ratio_es = ((np.nanmean(perc_change_es) - adjusted_risk_free_rate)/sigma) * np.sqrt(n_medio_trade_anno)
cagr = ((df_es_bt['Account'].iloc[-1]/df_es_bt['Account'].iloc[0])**(1/anni_totali)-1)*100

fig_sharpe = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=sharp_ratio_es,
    number={'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - Sharpe Ratio", 'font': {'size': 32}}
), row=1, col=1)
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=cagr,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % CAGR", 'font': {'size': 32}}
), row=1, col=2)

fig_sharpe.update_layout(plot_bgcolor="skyblue", paper_bgcolor="skyblue", font=dict(color="black"))
fig_sharpe.show()

# -----------------------------
# ----- 6. Rolling Average Units & Trade Count -----
# -----------------------------
df_es_bt_unit_list = df_es_bt.copy()
df_es_bt_unit_list['Rolling_Avg'] = df_es_bt_unit_list['Units'].iloc[1:].rolling(window=20).mean()
n_trades = len(df_es_bt.iloc[1:])

fig_units = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], specs=[[{"type": "xy"}, {"type": "domain"}]], subplot_titles=("Average Traded Units (20)", ""))

# Rolling average line
fig_units.add_trace(go.Scatter(
    x=df_es_bt_unit_list['Date'].iloc[20:], 
    y=df_es_bt_unit_list['Rolling_Avg'].iloc[20:], 
    mode='lines', 
    name=f'{strategy}'
), row=1, col=1)

# Number of trades indicator
fig_units.add_trace(go.Indicator(
    mode="number",
    value=n_trades,
    number={'valueformat': '.0f', 'font': {'size': 48}},
    title={'text': "Number of Trades", 'font': {'size': 32}}
), row=1, col=2)

fig_units.update_layout(plot_bgcolor="rgba(64,64,64,0.8)", paper_bgcolor="rgba(64,64,64,0.8)", font=dict(color="white"))
fig_units.show()

# -----------------------------
# ----- 7. Drawdown Statistics -----
# -----------------------------
# Funzione drawdowns_stats già definita
equity_curve_es_bt = df_es_bt['Account']
stats_A = drawdowns_stats(equity_curve_es_bt)

equity_curve_spy_buy_hold = df_spy_buy_hold['Account']
stats_B = drawdowns_stats(equity_curve_spy_buy_hold)

fig_dd = go.Figure()

# Strategia A
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{strategy} - Max DD", 'font':{'size':32}}, domain={'row':0,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':0,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':0,'column':2}))

# Strategia B
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{Strategy_B} - Max DD", 'font':{'size':32}}, domain={'row':1,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':1,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':1,'column':2}))

fig_dd.update_layout(
    grid={'rows':2,'columns':3},
    title=dict(text="Drawdown Statistics", font=dict(size=48)),
    height=500,
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white")
)

fig_dd.add_shape(
    type="line",
    x0=0, x1=1,       
    y0=0.55, y1=0.55,  
    xref="paper",
    yref="paper",
    line=dict(
        color="white",
        width=2))

fig_dd.show() 

## YM

### YM Data

In [ ]:
dia_connection_sql = mysql.connector.connect(
    host="localhost",
    user="root",
    password= f"{password_sql}",
    database="dia"
)

dia_db = pd.read_sql(
    'SELECT * FROM dia_historical_data1 WHERE DATE(date_ny) >= "2015-01-01" AND DATE(date_ny) < "2025-12-01"',
    dia_connection_sql
)
print(dia_db.dtypes)

dia_db['date'] = dia_db['date_ny'].dt.date
dia_db['time'] = dia_db['date_ny'].dt.time

dia_db = dia_db.sort_values(by=['date', 'time']).reset_index(drop=True)


In [ ]:
dia_buy_hold_open = dia_db['open'].iloc[0]
dia_buy_hold_units = round(100000 / dia_buy_hold_open, 0)
dia_buy_hold_close = dia_db.groupby(dia_db['date'])['close'].last()
dia_buy_hold_account = 100000 + (dia_buy_hold_close - dia_buy_hold_open) * dia_buy_hold_units
df_dia_buy_hold = pd.DataFrame(list(dia_buy_hold_account.items()), columns=['Date', 'Account'])


In [ ]:
ym_connection_sql = mysql.connector.connect(
    host="localhost",
    user="root",
    password= f"{password_sql}",
    database="ym"
)

ym_db = pd.read_sql(
    'SELECT * FROM ym_historical_data_5m_1 WHERE DATE(date_ny) >= "2015-01-01" AND DATE(date_ny) < "2025-12-01"',
    ym_connection_sql
)


In [ ]:
ym_connection_sql = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = f"{password_sql}",
    database = "ym")

ym_db = pd.read_sql('Select * from ym_historical_data_5m_1 where date(date_ny) >= "2015-01-01" and date(date_ny) < "2025-12-01"', ym_connection_sql)


In [ ]:
print(ym_db.dtypes)

ym_db['date'] = ym_db['date_ny'].dt.date
ym_db['time'] = ym_db['date_ny'].dt.time

ym_db = ym_db.sort_values(by = ['date','time']).reset_index(drop=True)


### YM Class

In [ ]:
class ym_rdr:
    def __init__(self):
        # --- Settings ---
        self.initial_account = 100000
        self.fee = 0.85     #Per part (no roundturn)
        self.point_value = 5
        self.tick_size = 1
        
        # --- Position state ---
        self.buy_pos = 0
        self.sell_pos = 0

        # --- Prices (theoretical) ---
        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        # --- Prices (real) ---
        self.real_buy_price = 0
        self.real_sell_price = 0

        # --- Trade info ---
        self.units = 0
        self.risk = 0
        self.account = []

        # --- Stats ---
        self.trades_pnl = []
        self.unit_list = []
        self.dates = []
        self.unit_trade = []
        self.real_buy_price_history = []
        self.real_sell_price_history = []
        self.direction_history = []

    # ===============================
    # POSITION SIZING (NO SLIPPAGE)
    # ===============================
    def compute_units(self):
        self.units = round(min((self.account[-1] * 0.01) / (self.risk*self.point_value),(4 * self.account[-1]) / (self.opening_price*self.point_value)),0)
        if self.units > 0:
            self.unit_list.append(self.units)

    # ======================================
    # WEIGHTED PRICE WITH STEP SLIPPAGE
    # ======================================
    def weighted_price(self, price, units, side, slip):
        remaining = units
        level = 0
        average_price = 0

        while remaining > 0:
            if level == 0:
                a = (5 - 12)/4 # Lim Inf Set  = 5
                initial_value = np.round(truncnorm(a, np.inf, loc = 12, scale = 4).rvs(),0)
                qty = min(initial_value, remaining)
            
            elif level == 1:
                value = round(np.round(uniform(loc = 0.6, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)
            
            elif level == 2:
                value = round(np.round(uniform(loc = 0.4, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            elif level >= 3:
                value = round(np.round(uniform(loc = 0.2, scale = 0.2).rvs(),1) * initial_value,0)
                qty = min(value, remaining)

            if side == 'Buy':
                exec_price = price + level * slip   # peggiora verso l'alto
            else:  # Sell
                exec_price = price - level * slip   # peggiora verso il basso

            average_price += qty * exec_price
            remaining -= qty
            level += 1

        return average_price / units

    # ===============================
    # ENTRY EXECUTION
    # ===============================
    def apply_entry(self, direction):
        self.compute_units()

        if self.units == 0:
            self.reset()
            return

        elif direction == 'Buy':
            self.real_buy_price = self.weighted_price(self.buy_price, self.units, side='Buy', slip = self.tick_size)
            self.direction_history.append('Buy')

        elif direction == 'Sell':
            self.real_sell_price = self.weighted_price(self.sell_price, self.units, side='Sell', slip = self.tick_size)
            self.direction_history.append('Sell')


    # ===============================
    # STOP EXECUTION 
    # ===============================
    def apply_stop(self, direction):
        if direction == 'Buy':  
            self.real_sell_price = self.weighted_price(
                self.sell_price, self.units, side='Sell', slip = self.tick_size
            )

        elif direction == 'Sell':  
            self.real_buy_price = self.weighted_price(
                self.buy_price, self.units, side='Buy', slip = self.tick_size
            )

    # ===============================
    # RECORD TRADE
    # ===============================
    def starting_backtest(self,date):
        self.dates.append(date)
        self.trades_pnl.append(0)
        self.account.append(self.initial_account)
        self.real_sell_price_history.append(0)
        self.real_buy_price_history.append(0)
        self.unit_trade.append(0)
        self.direction_history.append('None')
        self.unit_list.append(0)

    def record_trade(self, date):
        pnl = ((self.real_sell_price - self.real_buy_price) * self.units * self.point_value - 2 * self.units * self.fee)

        self.dates.append(date)
        self.trades_pnl.append(pnl)
        self.account.append(self.account[-1] + pnl)
        self.real_sell_price_history.append(self.real_sell_price)
        self.real_buy_price_history.append(self.real_buy_price)
        self.unit_trade.append(self.trades_pnl[-1]/self.units)


    # ===============================
    # RESET SINGLE TRADE
    # ===============================
    def reset(self):
        self.buy_pos = 0
        self.sell_pos = 0

        self.buy_price = 0
        self.sell_price = 0
        self.opening_price = 0
        self.stop_loss = 0
        self.take_profit = 0

        self.real_buy_price = 0
        self.real_sell_price = 0

        self.units = 0
        self.risk = 0

    # ===============================
    # TRADE LOG
    # =============================== 

    def trade_log(self):
        return pd.DataFrame({
            "Date": pd.to_datetime(self.dates),
            "Direction": self.direction_history,
            "Average_Buy": self.real_buy_price_history,
            "Average_Sell": self.real_sell_price_history,
            "Units": self.unit_list,
            "PnL": self.trades_pnl,
            "PnL_per_Unit": self.unit_trade,
            "Account": self.account}) 

### YM Backtest

In [ ]:
ym_bt = ym_rdr()

for x in tqdm(range(0, len(ym_db))):

    if x == 0:
        ym_bt.starting_backtest(ym_db['date'].iloc[x])

    # ================= ENTRY =================
    if ym_db['time'].iloc[x] == dt.time(9,35,0):

        # BUY
        if (ym_db['close'].iloc[x-1] - ym_db['open'].iloc[x-2]) > 0:
            ym_bt.buy_pos = 1
            ym_bt.buy_price = ym_db['open'].iloc[x]
            ym_bt.opening_price = ym_bt.buy_price
            ym_bt.stop_loss = min(ym_db['low'].iloc[x-1], ym_db['low'].iloc[x-2])
            ym_bt.take_profit = (ym_bt.opening_price - ym_bt.stop_loss) * 10 + ym_bt.opening_price
            ym_bt.risk = abs(ym_bt.opening_price - ym_bt.stop_loss)
            ym_bt.stop_to_be = False
            ym_bt.stop_step_1 = False

            ym_bt.apply_entry('Buy')
            real_risk = abs(ym_bt.real_buy_price - ym_bt.stop_loss)

        # SELL
        if (ym_db['close'].iloc[x-1] - ym_db['open'].iloc[x-2]) < 0:
            ym_bt.sell_pos = 1
            ym_bt.sell_price = ym_db['open'].iloc[x]
            ym_bt.opening_price = ym_bt.sell_price
            ym_bt.stop_loss = max(ym_db['high'].iloc[x-1], ym_db['high'].iloc[x-2])
            ym_bt.take_profit = (ym_bt.opening_price - ym_bt.stop_loss) * 10 + ym_bt.opening_price
            ym_bt.risk = abs(ym_bt.opening_price - ym_bt.stop_loss)
            ym_bt.stop_to_be = False
            ym_bt.stop_step_1 = False

            ym_bt.apply_entry('Sell')
            real_risk = abs(ym_bt.real_sell_price - ym_bt.stop_loss)


    # ================= LONG =================
    if ym_bt.buy_pos == 1 and ym_bt.sell_pos == 0:

        unrealized_profit = ym_db['close'].iloc[x] - ym_bt.real_buy_price

        # Stop loss Iniziale
        if ym_db['low'].iloc[x] <= ym_bt.stop_loss:
            ym_bt.sell_pos = 1
            ym_bt.sell_price = ym_bt.stop_loss
            ym_bt.apply_stop('Buy')

        # Take profit normale
        elif ym_db['high'].iloc[x] >= ym_bt.take_profit:
            ym_bt.sell_pos = 1
            ym_bt.sell_price = ym_bt.take_profit
            ym_bt.real_sell_price = ym_bt.sell_price  # NO slippage

        # ================= STOP MANAGEMENT =================
        elif (not ym_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            ym_bt.stop_loss = ym_bt.real_buy_price + 5 * real_risk
            ym_bt.stop_step_1 = True
            ym_bt.stop_to_be = True

        elif (not ym_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            ym_bt.stop_loss = ym_bt.real_buy_price  # stop a BE
            ym_bt.stop_to_be = True

    # ================= SHORT =================
    if ym_bt.buy_pos == 0 and ym_bt.sell_pos == 1:

        unrealized_profit = ym_bt.real_sell_price - ym_db['close'].iloc[x]

        # Stop loss iniziale
        if ym_db['high'].iloc[x] >= ym_bt.stop_loss:
            ym_bt.buy_pos = 1
            ym_bt.buy_price = ym_bt.stop_loss
            ym_bt.apply_stop('Sell')

        # Take profit normale
        elif ym_db['low'].iloc[x] <= ym_bt.take_profit:
            ym_bt.buy_pos = 1
            ym_bt.buy_price = ym_bt.take_profit
            ym_bt.real_buy_price = ym_bt.buy_price  # NO slippage

        # ================= STOP MANAGEMENT ================
        elif (not ym_bt.stop_step_1) and (unrealized_profit >= 6 * real_risk):
            ym_bt.stop_loss = ym_bt.real_sell_price - 5 * real_risk
            ym_bt.stop_step_1 = True
            ym_bt.stop_to_be = True

        elif (not ym_bt.stop_to_be) and (unrealized_profit >= 1 * real_risk):
            ym_bt.stop_loss = ym_bt.real_sell_price  # stop a BE
            ym_bt.stop_to_be = True

    # ================= END DAY =================
    if ym_db['time'].iloc[x] == dt.time(15,55,0):

        if ym_bt.buy_pos == 1 and ym_bt.sell_pos == 0:
            ym_bt.sell_pos = 1
            ym_bt.sell_price = ym_db['close'].iloc[x]
            ym_bt.real_sell_price = ym_db['close'].iloc[x]

        if ym_bt.buy_pos == 0 and ym_bt.sell_pos == 1:
            ym_bt.buy_pos = 1
            ym_bt.buy_price = ym_db['close'].iloc[x]
            ym_bt.real_buy_price = ym_db['close'].iloc[x]

    # ================= RECORD =================
    if ym_bt.buy_pos == 1 and ym_bt.sell_pos == 1:
        ym_bt.record_trade(ym_db['date'].iloc[x])
        ym_bt.reset()

### YM Results

In [ ]:
# Stategy Comparison
strategy = 'YM Trading Strategy'
Strategy_B = 'DIA Buy & Hold'

# -----------------------------
# ----- 1. Equity Curve -------
# -----------------------------
fig_eq = go.Figure()
fig_eq.add_trace(go.Scatter(x=ym_bt.dates, y=ym_bt.account, mode='lines', name=f'{strategy}'))
fig_eq.add_trace(go.Scatter(x=df_dia_buy_hold['Date'], y=df_dia_buy_hold['Account'], mode='lines', name=f'{Strategy_B}'))

fig_eq.update_layout(
    title=dict(text="Equity Curve", yanchor="top", font=dict(size=30)),
    margin=dict(l=50, r=50, t=70, b=50),
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    yaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)", zerolinewidth=1),
    xaxis=dict(showgrid=True, gridcolor="rgba(255,255,255,0.3)")
)
fig_eq.show()

# -----------------------------
# ----- 2. Trade Histogram -----
# -----------------------------
df_ym_bt = ym_bt.trade_log()
winning_ym_bt = df_ym_bt.loc[df_ym_bt['PnL'] > 0]
losing_ym_bt = df_ym_bt.loc[df_ym_bt['PnL'] < 0]

fig_hist = make_subplots(rows=1, cols=2, subplot_titles=("Winning Trades", "Losing Trades"))

# Istogramma vincite
fig_hist.add_trace(go.Histogram(
    x=winning_ym_bt['PnL_per_Unit'],
    nbinsx=50,
    marker_color='lightgreen',
    opacity=0.75
), row=1, col=1)
fig_hist.add_vline(x=np.mean(winning_ym_bt['PnL_per_Unit']),
                   line=dict(color="green", width=3, dash="dash"), row=1, col=1)

# Istogramma perdite
fig_hist.add_trace(go.Histogram(
    x=losing_ym_bt['PnL_per_Unit'],
    nbinsx=30,
    marker_color='lightcoral',
    opacity=0.75
), row=1, col=2)
fig_hist.add_vline(x=np.mean(losing_ym_bt['PnL_per_Unit']),
                   line=dict(color="red", width=3, dash="dash"), row=1, col=2)

fig_hist.update_layout(
    title=dict(text="Profit VS Loss per Contract", font=dict(size=48)),
    title_x=0.5,
    showlegend=False,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=1)
fig_hist.update_xaxes(tickformat=",~s", row=1, col=2)
fig_hist.show()

# -----------------------------
# ----- 3. Winrate -------------
# -----------------------------
p_win = len(winning_ym_bt) / len(df_ym_bt)
p_loss = 1 - p_win

fig_winrate = go.Figure()
fig_winrate.add_trace(go.Bar(
    x=['Win', 'Loss'],
    y=[p_win, p_loss],
    marker_color=['lightgreen', 'lightcoral'],
    opacity=0.85,
    showlegend=False,
    text=[f'{p_win*100:.1f}%', f'{p_loss*100:.1f}%'],
    textposition='outside'
))
fig_winrate.update_layout(
    title=dict(text="Winrate %", font=dict(size=48)),
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white"),
    bargap=0.15
)
fig_winrate.update_yaxes(range=[0, max(p_win, p_loss) + 0.1])
fig_winrate.show()

# -----------------------------
# ----- 4. Expected Value -----
# -----------------------------
EV_dollars_contract_ym = round(np.mean(df_ym_bt['PnL_per_Unit'].iloc[1:]), 2)
EV_perc_trade_ym = round(np.mean(df_ym_bt['Account'].pct_change())*100, 2)

fig_ev = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_dollars_contract_ym,
    number={'suffix': '$', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - $ EV per Unit", 'font': {'size': 32}}
), row=1, col=1)
fig_ev.add_trace(go.Indicator(
    mode="number",
    value=EV_perc_trade_ym,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % EV per Trade", 'font': {'size': 32}}
), row=1, col=2)

fig_ev.update_layout(plot_bgcolor="lightgreen", paper_bgcolor="lightgreen", font=dict(color="black"))
fig_ev.show()

# -----------------------------
# ----- 5. Sharpe & CAGR -------
# -----------------------------
perc_change_ym = df_ym_bt['Account'].pct_change()
sigma = np.nanstd(perc_change_ym, ddof=1)
n_medio_trade_anno = len(df_ym_bt[1:]) / (df_ym_bt['Date'].dt.year.max() - df_ym_bt['Date'].dt.year.min() + 1)
adjusted_risk_free_rate = (1 + 0.04)**(1/n_medio_trade_anno) - 1
delta_giorni = (df_ym_bt['Date'].max() - df_ym_bt['Date'].min()).days
anni_totali = delta_giorni / 365

sharp_ratio_ym = ((np.nanmean(perc_change_ym) - adjusted_risk_free_rate)/sigma) * np.sqrt(n_medio_trade_anno)
cagr = ((df_ym_bt['Account'].iloc[-1]/df_ym_bt['Account'].iloc[0])**(1/anni_totali)-1)*100

fig_sharpe = make_subplots(rows=1, cols=2, column_widths=[0.5, 0.5], specs=[[{"type": "indicator"}, {"type": "indicator"}]])
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=sharp_ratio_ym,
    number={'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - Sharpe Ratio", 'font': {'size': 32}}
), row=1, col=1)
fig_sharpe.add_trace(go.Indicator(
    mode="number",
    value=cagr,
    number={'suffix': '%', 'valueformat': '.2f', 'font': {'size': 48}},
    title={'text': f"{strategy} - % CAGR", 'font': {'size': 32}}
), row=1, col=2)

fig_sharpe.update_layout(plot_bgcolor="skyblue", paper_bgcolor="skyblue", font=dict(color="black"))
fig_sharpe.show()

# -----------------------------
# ----- 6. Rolling Average Units & Trade Count -----
# -----------------------------
df_ym_bt_unit_list = df_ym_bt.copy()
df_ym_bt_unit_list['Rolling_Avg'] = df_ym_bt_unit_list['Units'].iloc[1:].rolling(window=20).mean()
n_trades = len(df_ym_bt.iloc[1:])

fig_units = make_subplots(rows=1, cols=2, column_widths=[0.7, 0.3], specs=[[{"type": "xy"}, {"type": "domain"}]], subplot_titles=("Average Traded Units (20)", ""))

# Rolling average line
fig_units.add_trace(go.Scatter(
    x=df_ym_bt_unit_list['Date'].iloc[20:], 
    y=df_ym_bt_unit_list['Rolling_Avg'].iloc[20:], 
    mode='lines', 
    name=f'{strategy}'
), row=1, col=1)

# Number of trades indicator
fig_units.add_trace(go.Indicator(
    mode="number",
    value=n_trades,
    number={'valueformat': '.0f', 'font': {'size': 48}},
    title={'text': "Number of Trades", 'font': {'size': 32}}
), row=1, col=2)

fig_units.update_layout(plot_bgcolor="rgba(64,64,64,0.8)", paper_bgcolor="rgba(64,64,64,0.8)", font=dict(color="white"))
fig_units.show()

# -----------------------------
# ----- 7. Drawdown Statistics -----
# -----------------------------
# Funzione drawdowns_stats già definita
equity_curve_ym_bt = df_ym_bt['Account']
stats_A = drawdowns_stats(equity_curve_ym_bt)

equity_curve_dia_buy_hold = df_dia_buy_hold['Account']
stats_B = drawdowns_stats(equity_curve_dia_buy_hold)

fig_dd = go.Figure()

# Strategia A
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{strategy} - Max DD", 'font':{'size':32}}, domain={'row':0,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':0,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_A['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':0,'column':2}))

# Strategia B
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Max Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':f"{Strategy_B} - Max DD", 'font':{'size':32}}, domain={'row':1,'column':0}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average Drawdown']*100, number={'suffix':'%', 'valueformat':'.2f','font':{'size':48}}, title={'text':"Average DD",'font':{'size':32}}, domain={'row':1,'column':1}))
fig_dd.add_trace(go.Indicator(mode="number", value=stats_B['Average DD Duration'], number={'valueformat':'.1f','font':{'size':48}}, title={'text':"DD Duration (Trades)",'font':{'size':32}}, domain={'row':1,'column':2}))

fig_dd.update_layout(
    grid={'rows':2,'columns':3},
    title=dict(text="Drawdown Statistics", font=dict(size=48)),
    height=500,
    title_x=0.5,
    plot_bgcolor="rgba(64,64,64,0.8)",
    paper_bgcolor="rgba(64,64,64,0.8)",
    font=dict(color="white")
)

fig_dd.add_shape(
    type="line",
    x0=0, x1=1,       
    y0=0.55, y1=0.55,  
    xref="paper",
    yref="paper",
    line=dict(
        color="white",
        width=2))

fig_dd.show()
